In [1]:
!kinit -kt $KRB5_CLIENT_KTNAME $KRB5_PRINCIPAL

In [1]:
%%configure -f 
{
    "conf":
    {
        "spark.app.name":"Bhavesh_Inventory_Metrics",
        "spark.yarn.queue": "default",
        "spark.jars": "/apps/Jars/obelisk-retail-legos-Bhavesh.jar,/apps/Jars/obelisk-retail-usecases-Bhavesh.jar,/apps/Jars/mysql-connector-java-5.1.40.jar",
        "spark.driver.extraJavaOptions": "-Dscala.color",
        "spark.driver.memory": "3g",
        "spark.executor.memory": "3g",
        "spark.executor.instances": "4"
    }
}

In [2]:
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.Constants.STANDARD_COL_NAMES._
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.io._
import ai.couture.obelisk.commons.utils.DateTimeUtil._
import org.apache.spark.sql._
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import spark.implicits._

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1392,application_1686980170466_0031,spark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.Constants.STANDARD_COL_NAMES._
import ai.couture.obelisk.commons.Constants._
import ai.couture.obelisk.commons.io._
import ai.couture.obelisk.commons.utils.DateTimeUtil._
import org.apache.spark.sql._
import org.apache.spark.sql.expressions._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.types._
import spark.implicits._


### Local Metrics

In [6]:
def thresholdColumns(columns: Array[String], minThreshold: Double = 0, maxThreshold: Double = Double.PositiveInfinity)(df: DataFrame): DataFrame = {

    var scaledDF = df
    columns.foreach(
    column => {
    scaledDF = scaledDF.withColumn(column,
      when(col(column) > maxThreshold, lit(maxThreshold)).otherwise(col(column)))
      .withColumn(column, when(col(column) < minThreshold, lit(minThreshold)).otherwise(col(column)))
    })
    scaledDF
}

var runDate: String = "2023-04-30"
var baseDir: String = "/data/Archive/bhavesh/InventoryPrediction"
var runDir: String = s"$baseDir/$runDate"
var suffix: String = "colorfamily_pricebucket_styletype_pattern_sleeve_brandname"
var prodLevelSigma = ParquetToDF.getDF(s"$runDir/Statistics12Months/product").select(PRODUCTID, "stddev")
var queryLevelSigma = ParquetToDF.getDF(s"$runDir/Statistics12Months/query").select("query", "stddev", SIMILAR_GROUP_LEVEL, "template")
var productQueryMap = ParquetToDF.getDF(s"$runDir/QueryProductMapping/suffix=$suffix").filter(col("query") =!= "")

def convertLocalToRequiredFormat(dateForWhichPredictionsAreMade: String, basePath: String, level: String, localFile: String, fileType: String = "csv", model: String = "lr"): Unit = {
    
    var actual = ParquetToDF.getDF(s"/data/Archive/bhavesh/InventoryPrediction/actualDataForPredictionPeriod/date=$dateForWhichPredictionsAreMade/sales")
    var fileName: String = if(level == "product"){
       "productLevelPredictions"
    } else{
        "queryAggregatedPredictions"
    }
    def doConversion(brick: String, brickId: String): DataFrame = {
        var localPredictionsPath = s"$basePath/$brick/$localFile"
        var df = if(fileType=="csv"){
            CSVToDF.getDF(localPredictionsPath, inferSchema = true)
        }
        else{
            ParquetToDF.getDF(localPredictionsPath)
        }

        
        df = df
        .withColumn(SIMILAR_GROUP_LEVEL, lit(brickId))
        .withColumn("model", lit(model))
        .withColumn("predicted", col("predictedyQuantity"))
        .select(PRODUCTID, SIMILAR_GROUP_LEVEL, "predicted", "model")
        
        if(level == "product") {
            df.transform(thresholdColumns(Array("predicted")))
            .join(prodLevelSigma, Seq(PRODUCTID))
            .withColumn("lowerBound", col("predicted") - (lit(2.0) * col("stddev")))
            .withColumn("upperBound", col("predicted") + (lit(2.0) * col("stddev")))
            .transform(thresholdColumns(Array("lowerBound")))
        }
        else {
            df.transform(thresholdColumns(Array("predicted")))
            .join(productQueryMap, Seq(SIMILAR_GROUP_LEVEL, PRODUCTID))
            .groupBy("model", "query", SIMILAR_GROUP_LEVEL, "template")
            .agg(sum("predicted") as "predicted")
            .join(queryLevelSigma, Seq(SIMILAR_GROUP_LEVEL, "query", "template"))
            .withColumn("lowerBound", col("predicted") - (lit(2.0) * col("stddev")))
            .withColumn("upperBound", col("predicted") + (lit(2.0) * col("stddev")))
            .transform(thresholdColumns(Array("lowerBound")))
        }
    }
    var menShirts = doConversion("menShirts", "830216013")
    DFToParquet.putDF(s"$basePath/menShirts/$fileName", menShirts, partitionedColumn="model")
    var womenKurtas = doConversion("womenKurtas", "830303011")
    DFToParquet.putDF(s"$basePath/womenKurtas/$fileName", womenKurtas, partitionedColumn="model")
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

thresholdColumns: (columns: Array[String], minThreshold: Double, maxThreshold: Double)(df: org.apache.spark.sql.DataFrame)org.apache.spark.sql.DataFrame
runDate: String = 2023-04-30
baseDir: String = /data/Archive/bhavesh/InventoryPrediction
runDir: String = /data/Archive/bhavesh/InventoryPrediction/2023-04-30
suffix: String = colorfamily_pricebucket_styletype_pattern_sleeve_brandname
prodLevelSigma: org.apache.spark.sql.DataFrame = [productid: string, stddev: double]
queryLevelSigma: org.apache.spark.sql.DataFrame = [query: string, stddev: double ... 2 more fields]
productQueryMap: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [query: string, productid: string ... 2 more fields]
convertLocalToRequiredFormat: (dateForWhichPredictionsAreMade: String, basePath: String, level: String, localFile: String, fileType: String, model: String)Unit


In [38]:
def convertLocal(exp: String, date: String, filename: String, format: String, baseDir: String, model: String = "lr"){
    convertLocalToRequiredFormat(date, s"$baseDir/experiments/exp_$exp/predictions/ModelForEachBrickProductLevel", "product", filename, format, model = model)
    convertLocalToRequiredFormat(date, s"$baseDir/experiments/exp_$exp/predictions/ModelForEachBrickProductLevel", "query", filename, format, model = model)
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

convertLocal: (exp: String, date: String, filename: String, format: String, baseDir: String, model: String)Unit


In [4]:
def convertLocal2(exp: String, date: String, filename: String, format: String, baseDir: String, model: String = "lr"){
    convertLocalToRequiredFormat(date, s"$baseDir/NMonthsAheadExperiment/${exp}MonthsAhead/predictions", "product", filename, format, model = model)
    convertLocalToRequiredFormat(date, s"$baseDir/NMonthsAheadExperiment/${exp}MonthsAhead/predictions", "query", filename, format, model = model)
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

convertLocal2: (exp: String, date: String, filename: String, format: String, baseDir: String, model: String)Unit


In [9]:
convertLocal2(
    "6",
    date = "2023-05-01",
    filename = "predictions",
    format = "parquet",
    baseDir = "/data/Archive/bhavesh/InventoryPrediction/2023-04-30",
    model = "neural_network"
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [41]:
convertLocal(
    exp = "neural_network_dropout_lr_0.0005",
    date = "2023-05-01",
    filename = "predictions",
    format = "parquet",
    baseDir = "/data/Archive/bhavesh/InventoryPrediction/2023-04-30",
    model = "neural_network"
)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Metrics Show

In [12]:
def metricsShow(exp: String): Unit = {
    var pl = CSVToDF.getDF(s"/data/Archive/bhavesh/InventoryPrediction/2023-04-30/experiments/exp_${exp}/metrics/productLevelModelForEachBrick/productLevel", inferSchema=true).orderBy("model", "similargrouplevel").drop(DATE).persist()
    pl.printSchema

    var show = pl.orderBy("metric").groupBy("model", "metric").pivot("similargrouplevel").agg(first("value")).orderBy("model", "metric").withColumn("final", concat(lit("Men Shirts: "), col("830216013"), lit("\n"), lit("Women Kurtas: "), col("830303011"))).groupBy("model").pivot("metric").agg(first("final") as "final").persist()
    show.orderBy("model").show(false)
    // show.orderBy("model").drop("model").show(false)
    DFToCSV.putDF("/data/Archive/bhavesh/InventoryPrediction/temp/ip/pl_show", show.orderBy("model").coalesce(1))
    pl.unpersist()

    var ql = CSVToDF.getDF(s"/data/Archive/bhavesh/InventoryPrediction/2023-04-30/experiments/exp_${exp}/metrics/productLevelModelForEachBrick/queryLevel", inferSchema=true).orderBy("model", "similargrouplevel").drop(DATE).persist()
    ql.printSchema

    show = ql.orderBy("metric").groupBy("model", "metric").pivot("similargrouplevel").agg(first("value")).orderBy("model", "metric").withColumn("final", concat(lit("Men Shirts: "), col("830216013"), lit("\n"), lit("Women Kurtas: "), col("830303011"))).groupBy("model").pivot("metric").agg(first("final") as "final").persist()
    show.orderBy("model").show(false)
    // show.orderBy("model").drop("model").show(false)
    DFToCSV.putDF("/data/Archive/bhavesh/InventoryPrediction/temp/ip/ql_show", show.orderBy("model").coalesce(1))
    ql.unpersist()
}

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

metricsShow: (exp: String)Unit


In [28]:
var ql = CSVToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/experiments/combined/nn_crostonsba/metrics/date=2023-05-01").orderBy("similargrouplevel").persist()
ql.printSchema
var show = ql.groupBy("metric").pivot("similargrouplevel").agg(first("value")).orderBy("metric").withColumn("final", concat(lit("Men Shirts: "), col("830216013"), lit("\n"), lit("Women Kurtas: "), col("830303011"))).withColumn("model", lit("combined")).groupBy("model").pivot("metric").agg(first("final") as "final").persist()
show.show()
DFToCSV.putDF("/data/Archive/bhavesh/InventoryPrediction/temp/ip/ql_show", show.coalesce(1))
ql.unpersist()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

ql: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [metric: string, value: string ... 1 more field]
root
 |-- metric: string (nullable = true)
 |-- value: string (nullable = true)
 |-- similargrouplevel: integer (nullable = true)

show: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [model: string, Bias: string ... 7 more fields]
+--------+--------------------+--------------------+---------------------------+-------------------------------------------------+------------------------------------------------+------------------------------------------------+------------------------------------------------+------------------------------------------------+
|   model|                Bias|            Hit Rate|Mean Absolute Percent Error|Root Mean Square Error for 100.0 percentile error|Root Mean Square Error for 25.0 percentile error|Root Mean Square Error for 50.0 percentile error|Root Mean Square Error for 75.0 percentile error|Root Mean Square Error for 90.0 percentile e

In [13]:
val exp_code = "1"
metricsShow(exp_code)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

exp_code: String = 1
root
 |-- metric: string (nullable = true)
 |-- value: double (nullable = true)
 |-- model: string (nullable = true)
 |-- similargrouplevel: integer (nullable = true)

+-----+----------------------------------------------------------------+---------------------------------------------------------------+---------------------------------------------------------------+--------------------------------------------------------------+---------------------------------------------------------------+--------------------------------------------------------------+--------------------------------------------------------------+---------------------------------------------------------------+
|model|Bias                                                            |Hit Rate                                                       |Mean Absolute Percent Error                                    |Root Mean Square Error for 100.0 percentile error             |Root Mean Square Error for 25.

### Combined 6 months predictions to one file

In [11]:
var basePath: String = "/data/Archive/bhavesh/InventoryPrediction/2023-04-30/NMonthsAheadExperiment"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

basePath: String = /data/Archive/bhavesh/InventoryPrediction/2023-04-30/NMonthsAheadExperiment


In [14]:
var oldQueryToNewQueryMap = ParquetToDF.getDF("/data/Archive/inventory/2023-04-30/queryProductMap_colorfamily_pricebucket_styletype_pattern_sleeve_brandname/attributesQueryInfo/final").select("query","template","similargrouplevel","newQuery")
oldQueryToNewQueryMap.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

oldQueryToNewQueryMap: org.apache.spark.sql.DataFrame = [query: string, template: string ... 2 more fields]
root
 |-- query: string (nullable = true)
 |-- template: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- newQuery: string (nullable = true)



In [17]:
Array(1, 2, 3, 4, 5, 6).foreach(months => {
    var predictionsPath = s"$basePath/${months}MonthsAhead/predictions"
    var menShirts = ParquetToDF.getDF(s"$predictionsPath/menShirts/queryAggregatedPredictions/model=neural_network")
    .select("similargrouplevel", "query", "template", "predicted")
    var womenKurtas = ParquetToDF.getDF(s"$predictionsPath/womenKurtas/queryAggregatedPredictions/model=neural_network")
    .select("similargrouplevel", "query", "template", "predicted")
    
    var predictions = menShirts.union(womenKurtas).join(oldQueryToNewQueryMap, Seq("similargrouplevel", "query", "template"))
    .withColumnRenamed("query", "oldQuery")
    .withColumnRenamed("newQuery", "query")
    .select("similargrouplevel", "query", "predicted", "oldQuery", "template")
    
    DFToParquet.putDF(s"/data/Archive/bhavesh/InventoryPrediction/2023-04-30/NMonthsAheadExperiment/predictions/months=m$months", predictions)
})

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
var df = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/NMonthsAheadExperiment/predictions")
df.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df: org.apache.spark.sql.DataFrame = [similargrouplevel: string, query: string ... 4 more fields]
root
 |-- similargrouplevel: string (nullable = true)
 |-- query: string (nullable = true)
 |-- predicted: double (nullable = true)
 |-- oldQuery: string (nullable = true)
 |-- template: string (nullable = true)
 |-- months: string (nullable = true)



In [39]:
df.groupBy("similargrouplevel", "months").agg(count("query")).show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------+------------+
|similargrouplevel|months|count(query)|
+-----------------+------+------------+
|830303011        |m1    |286269      |
|830303011        |m3    |286269      |
|830303011        |m6    |286269      |
|830303011        |m2    |286269      |
|830216013        |m5    |425838      |
|830303011        |m5    |286269      |
|830216013        |m4    |425838      |
|830216013        |m1    |425838      |
|830303011        |m4    |286269      |
|830216013        |m2    |425838      |
|830216013        |m6    |425838      |
|830216013        |m3    |425838      |
+-----------------+------+------------+



In [34]:
var arima = ParquetToDF.getDF("/data/Archive/inventory/2023-04-30/queryProductMap_colorfamily_pricebucket_styletype_pattern_sleeve_brandname/allTimeSeriesFull6months/queryPredictions/model=DynamicOptimizedTheta")
arima.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

arima: org.apache.spark.sql.DataFrame = [query: string, predictionMonth: string ... 2 more fields]
root
 |-- query: string (nullable = true)
 |-- predictionMonth: string (nullable = true)
 |-- predicted: double (nullable = true)
 |-- similargrouplevel: integer (nullable = true)



In [41]:
arima.groupBy("similargrouplevel", "predictionMonth").agg(count("query")).show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+---------------+------------+
|similargrouplevel|predictionMonth|count(query)|
+-----------------+---------------+------------+
|830303011        |2023-09-01     |311129      |
|830303011        |2023-06-01     |311129      |
|830216013        |2023-07-01     |454092      |
|830216013        |2023-08-01     |454092      |
|830303011        |2023-08-01     |311129      |
|830303011        |2023-07-01     |311129      |
|830216013        |2023-05-01     |454092      |
|830216013        |2023-09-01     |454092      |
|830216013        |2023-06-01     |454092      |
|830303011        |2023-05-01     |311129      |
|830216013        |2023-10-01     |454092      |
|830303011        |2023-10-01     |311129      |
+-----------------+---------------+------------+



In [49]:
var monthDateToLetterMap = arima.select("predictionMonth").distinct().withColumn("monthInWords", concat(lit("m"), row_number().over(Window.orderBy(asc("predictionMonth"))))).persist()
monthDateToLetterMap.show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

monthDateToLetterMap: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [predictionMonth: string, monthInWords: string]
+---------------+------------+
|predictionMonth|monthInWords|
+---------------+------------+
|2023-05-01     |m1          |
|2023-06-01     |m2          |
|2023-07-01     |m3          |
|2023-08-01     |m4          |
|2023-09-01     |m5          |
|2023-10-01     |m6          |
+---------------+------------+



In [50]:
df = df.withColumn("model", lit("nn")).select(col("query"), col("similargrouplevel"), col("months").as("predictionMonth"), col("model"), col("predicted"))
arima = arima.withColumn("model", lit("arima")).join(monthDateToLetterMap, Seq("predictionMonth")).drop("predictionMonth").select(col("query"), col("similargrouplevel"), col("monthInWords").as("predictionMonth"), col("model"), col("predicted"))

df.printSchema
arima.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

df: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: string ... 3 more fields]
arima: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: int ... 3 more fields]
root
 |-- query: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- predictionMonth: string (nullable = true)
 |-- model: string (nullable = false)
 |-- predicted: double (nullable = true)

root
 |-- query: string (nullable = true)
 |-- similargrouplevel: integer (nullable = true)
 |-- predictionMonth: string (nullable = true)
 |-- model: string (nullable = false)
 |-- predicted: double (nullable = true)



In [51]:
arima.groupBy("similargrouplevel", "predictionMonth").agg(count("query")).show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+---------------+------------+
|similargrouplevel|predictionMonth|count(query)|
+-----------------+---------------+------------+
|830303011        |m5             |311129      |
|830216013        |m3             |454092      |
|830303011        |m2             |311129      |
|830303011        |m6             |311129      |
|830216013        |m1             |454092      |
|830216013        |m5             |454092      |
|830303011        |m1             |311129      |
|830216013        |m2             |454092      |
|830303011        |m4             |311129      |
|830216013        |m6             |454092      |
|830216013        |m4             |454092      |
|830303011        |m3             |311129      |
+-----------------+---------------+------------+



In [52]:
var combinedPredictions = df.union(arima).groupBy("similargrouplevel", "query", "predictionMonth").agg(avg("predicted") as "predicted")
combinedPredictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

combinedPredictions: org.apache.spark.sql.DataFrame = [similargrouplevel: string, query: string ... 2 more fields]
root
 |-- similargrouplevel: string (nullable = true)
 |-- query: string (nullable = true)
 |-- predictionMonth: string (nullable = true)
 |-- predicted: double (nullable = true)



In [53]:
DFToParquet.putDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/NMonthsAheadExperiment/combinedPredictions/nn_DynamicOptimizedTheta", combinedPredictions)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
var combinedPredictions = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/NMonthsAheadExperiment/combinedPredictions/nn_DynamicOptimizedTheta")
combinedPredictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

combinedPredictions: org.apache.spark.sql.DataFrame = [similargrouplevel: string, query: string ... 2 more fields]
root
 |-- similargrouplevel: string (nullable = true)
 |-- query: string (nullable = true)
 |-- predictionMonth: string (nullable = true)
 |-- predicted: double (nullable = true)



In [4]:
combinedPredictions.groupBy("similargrouplevel", "predictionMonth").agg(count("query")).show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+---------------+------------+
|similargrouplevel|predictionMonth|count(query)|
+-----------------+---------------+------------+
|830303011        |m1             |311129      |
|830303011        |m3             |311129      |
|830303011        |m6             |311129      |
|830303011        |m2             |311129      |
|830216013        |m5             |454092      |
|830303011        |m5             |311129      |
|830216013        |m4             |454092      |
|830216013        |m1             |454092      |
|830216013        |m2             |454092      |
|830216013        |m6             |454092      |
|830303011        |m4             |311129      |
|830216013        |m3             |454092      |
+-----------------+---------------+------------+



In [6]:
var runDir = "/data/Archive/bhavesh/InventoryPrediction/2023-04-30"
var productsActual = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/actualDataForPredictionPeriod/date=2023-05-01/sales")
var suffix = "colorfamily_pricebucket_styletype_pattern_sleeve_brandname"
var queriesToProductMap = ParquetToDF.getDF(s"$runDir/QueryProductMapping/suffix=$suffix").select("query", "productid", "similargrouplevel", "template").na.drop()
var oldQueryToNewQueryMap = ParquetToDF.getDF("/data/Archive/inventory/2023-04-30/queryProductMap_colorfamily_pricebucket_styletype_pattern_sleeve_brandname/attributesQueryInfo/final").select("query","template","similargrouplevel","newQuery")

queriesToProductMap = queriesToProductMap.join(oldQueryToNewQueryMap, Seq("similargrouplevel", "query", "template")).drop("query").withColumnRenamed("newQuery", "query")

var productAttr = (
    ParquetToDF.getDF("/data/Archive/inventory/"+"2023-04-30/queryProductMap_colorfamily_pricebucket_styletype_pattern_sleeve_brandname"+"/processedProductAttr")
    .select(PRODUCTID, SIMILAR_GROUP_LEVEL)
    .na.drop
)

productsActual = (
    productsActual      
    .join(productAttr, Seq(PRODUCTID))
    .select(PRODUCTID, "actual")
)

var queryActual = productsActual.join(queriesToProductMap, Seq(PRODUCTID)).groupBy("similargrouplevel", "query").agg(sum("actual") as "actual")
queryActual.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

runDir: String = /data/Archive/bhavesh/InventoryPrediction/2023-04-30
productsActual: org.apache.spark.sql.DataFrame = [productid: string, actual: bigint]
suffix: String = colorfamily_pricebucket_styletype_pattern_sleeve_brandname
queriesToProductMap: org.apache.spark.sql.DataFrame = [query: string, productid: string ... 2 more fields]
oldQueryToNewQueryMap: org.apache.spark.sql.DataFrame = [query: string, template: string ... 2 more fields]
queriesToProductMap: org.apache.spark.sql.DataFrame = [similargrouplevel: int, template: string ... 2 more fields]
productAttr: org.apache.spark.sql.DataFrame = [productid: string, similargrouplevel: string]
productsActual: org.apache.spark.sql.DataFrame = [productid: string, actual: bigint]
queryActual: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string ... 1 more field]
root
 |-- similargrouplevel: integer (nullable = true)
 |-- query: string (nullable = true)
 |-- actual: long (nullable = true)



In [7]:
DFToParquet.putDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/NMonthsAheadExperiment/actual", queryActual)
queryActual = ParquetToDF.getDF("/data/Archive/bhavesh/InventoryPrediction/2023-04-30/NMonthsAheadExperiment/actual")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

queryActual: org.apache.spark.sql.DataFrame = [similargrouplevel: int, query: string ... 1 more field]


In [8]:
var predictions = (
    combinedPredictions.groupBy("similargrouplevel", "query")
    .pivot("predictionMonth")
    .agg(sum("predicted"))
    .join(queryActual, Seq("query", "similargrouplevel"), "left")
    .withColumn("actual", col("actual").cast(DoubleType))
    .na.fill(0.0, Seq("actual"))
)
predictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

predictions: org.apache.spark.sql.DataFrame = [query: string, similargrouplevel: string ... 7 more fields]
root
 |-- query: string (nullable = true)
 |-- similargrouplevel: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)
 |-- actual: double (nullable = false)



In [9]:
predictions.groupBy("similargrouplevel").count.show

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------+
|similargrouplevel| count|
+-----------------+------+
|        830216013|454092|
|        830303011|311129|
+-----------------+------+



In [10]:
predictions.na.drop.groupBy("similargrouplevel").count.show

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------+
|similargrouplevel| count|
+-----------------+------+
|        830216013|454092|
|        830303011|311129|
+-----------------+------+



In [13]:
predictions = predictions.withColumn("category_name", when(col("similargrouplevel") === "830216013", lit("Men-Shirts")).otherwise(lit("Women-Kurtas")))
predictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

predictions: org.apache.spark.sql.DataFrame = [similargrouplevel: string, category_name: string ... 8 more fields]
root
 |-- similargrouplevel: string (nullable = true)
 |-- category_name: string (nullable = false)
 |-- query: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)
 |-- actual: double (nullable = false)



In [12]:
predictions = predictions.select("similargrouplevel", "category_name", "query", "m1", "m2", "m3", "m4", "m5", "m6", "actual")
predictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

predictions: org.apache.spark.sql.DataFrame = [similargrouplevel: string, category_name: string ... 8 more fields]
root
 |-- similargrouplevel: string (nullable = true)
 |-- category_name: string (nullable = false)
 |-- query: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)
 |-- actual: double (nullable = false)



In [14]:
DFToParquet.putDF("/data/ecomm/ajio/processed/inventory/integrationSimulator/predictions", predictions)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
var predictions = ParquetToDF.getDF("/data/ecomm/ajio/processed/inventory/integrationSimulator/predictions")
predictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

predictions: org.apache.spark.sql.DataFrame = [similargrouplevel: string, category_name: string ... 8 more fields]
root
 |-- similargrouplevel: string (nullable = true)
 |-- category_name: string (nullable = true)
 |-- query: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)
 |-- actual: double (nullable = true)



In [16]:
predictions.na.drop.groupBy("similargrouplevel").count.show

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+------+
|similargrouplevel| count|
+-----------------+------+
|        830216013|454092|
|        830303011|311129|
+-----------------+------+



In [22]:
predictions.show(5, false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-------------+--------------------------------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------+
|similargrouplevel|category_name|query                                       |m1                 |m2                 |m3                 |m4                 |m5                 |m6                 |actual|
+-----------------+-------------+--------------------------------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-------------------+------+
|830216013        |Men-Shirts   |bossini_pink_solid_2_#_classic              |9.482614755630493  |11.17599630355835  |12.983623385429382 |12.771461009979248 |19.449276208877563 |10.994883179664612 |3.0   |
|830216013        |Men-Shirts   |arrow-sports_red_check_2_#_classic          |143.1906315088272  |204.89316129684448 |201.19756412506104 |197.51847159862518 |207.42465138435364

In [23]:
predictions.filter(col("m1")>1000).show(5, false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+-------------+--------------------------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------+
|similargrouplevel|category_name|query                                             |m1                |m2                |m3                |m4                |m5                |m6                |actual |
+-----------------+-------------+--------------------------------------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------+
|830216013        |Men-Shirts   |the-indian-garage-co_navy_#_2_full-length sleeve_#|16037.080436170101|16930.395792484283|16545.689509868622|17657.16510117054 |17583.962018013   |16794.849339723587|13581.0|
|830303011        |Women-Kurtas |#_#_stripes_1_3-4th sleeve_straight               |8213.88771790266  |7903.296661674976 |8242.566283345222 |8104.708490490913 |8223.2044074

### Metrics Computation for the generated predictions of m1

In [3]:
var predictions = ParquetToDF.getDF("/data/ecomm/ajio/processed/inventory/integrationSimulator/predictions")
predictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

predictions: org.apache.spark.sql.DataFrame = [similargrouplevel: string, category_name: string ... 8 more fields]
root
 |-- similargrouplevel: string (nullable = true)
 |-- category_name: string (nullable = true)
 |-- query: string (nullable = true)
 |-- m1: double (nullable = true)
 |-- m2: double (nullable = true)
 |-- m3: double (nullable = true)
 |-- m4: double (nullable = true)
 |-- m5: double (nullable = true)
 |-- m6: double (nullable = true)
 |-- actual: double (nullable = true)



In [7]:
var oldQueryToNewQueryMap = ParquetToDF.getDF("/data/Archive/inventory/2023-04-30/queryProductMap_colorfamily_pricebucket_styletype_pattern_sleeve_brandname/attributesQueryInfo/final").select("query","template","similargrouplevel","newQuery")
queryLevelSigma = queryLevelSigma.join(oldQueryToNewQueryMap, Seq("similargrouplevel", "query", "template")).drop("query", "template").withColumnRenamed("newQuery", "query")
queryLevelSigma.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

oldQueryToNewQueryMap: org.apache.spark.sql.DataFrame = [query: string, template: string ... 2 more fields]
queryLevelSigma: org.apache.spark.sql.DataFrame = [similargrouplevel: int, stddev: double ... 1 more field]
root
 |-- similargrouplevel: integer (nullable = true)
 |-- stddev: double (nullable = true)
 |-- query: string (nullable = true)



In [8]:
predictions = predictions.select(col("similargrouplevel"), col("query"), col("m1") as "predicted", col("actual"))

predictions = (
    predictions.join(queryLevelSigma, Seq("similargrouplevel", "query"))
    .withColumn("lowerBound", col("predicted") - (lit(2.0) * col("stddev")))
    .withColumn("upperBound", col("predicted") + (lit(2.0) * col("stddev")))
    .transform(thresholdColumns(Array("lowerBound")))
)

predictions.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

predictions: org.apache.spark.sql.DataFrame = [similargrouplevel: string, query: string ... 2 more fields]
predictions: org.apache.spark.sql.DataFrame = [similargrouplevel: string, query: string ... 5 more fields]
root
 |-- similargrouplevel: string (nullable = true)
 |-- query: string (nullable = true)
 |-- predicted: double (nullable = true)
 |-- actual: double (nullable = true)
 |-- stddev: double (nullable = true)
 |-- lowerBound: double (nullable = true)
 |-- upperBound: double (nullable = true)



In [13]:
predictions.count

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

res21: Long = 765221


In [11]:
import ai.couture.obelisk.retail.usecases.inventoryprediction.Metrics.computeAllMetrics

var metricsMS = predictions.filter(col("similargrouplevel") === "830216013").transform(computeAllMetrics)
var metricsWK = predictions.filter(col("similargrouplevel") === "830303011").transform(computeAllMetrics)
metricsMS.printSchema
metricsWK.printSchema

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

import ai.couture.obelisk.retail.usecases.inventoryprediction.Metrics.computeAllMetrics
metricsMS: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [metric: string, value: double]
metricsWK: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [metric: string, value: double]
root
 |-- metric: string (nullable = true)
 |-- value: double (nullable = false)

root
 |-- metric: string (nullable = true)
 |-- value: double (nullable = false)



In [12]:
println("MenShirts")
metricsMS.show(false)
println("\nWomenKurtas")
metricsWK.show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

MenShirts
+-------------------------------------------------+--------------------+
|metric                                           |value               |
+-------------------------------------------------+--------------------+
|Mean Absolute Percent Error                      |216.87214674162075  |
|Bias                                             |1.0341494563089509E7|
|Hit Rate                                         |0.6030694220554426  |
|Root Mean Square Error for 25.0 percentile error |0.7321321348462142  |
|Root Mean Square Error for 50.0 percentile error |1.2777653048670148  |
|Root Mean Square Error for 75.0 percentile error |3.1086104050274006  |
|Root Mean Square Error for 90.0 percentile error |8.96610211138317    |
|Root Mean Square Error for 100.0 percentile error|400.190129625687    |
+-------------------------------------------------+--------------------+


WomenKurtas
+-------------------------------------------------+------------------+
|metric                      

In [14]:
println("Only for queries that had observed sales on test month")
println("\nMenShirts")
predictions.filter(col("similargrouplevel") === "830216013").filter(col("actual") > 0).transform(computeAllMetrics).show(false)
println("\nWomenKurtas")
predictions.filter(col("similargrouplevel") === "830303011").filter(col("actual") > 0).transform(computeAllMetrics).show(false)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Only for queries that had observed sales on test month

MenShirts
+-------------------------------------------------+------------------+
|metric                                           |value             |
+-------------------------------------------------+------------------+
|Mean Absolute Percent Error                      |216.87214674162075|
|Bias                                             |9748222.00341982  |
|Hit Rate                                         |0.7419049197183877|
|Root Mean Square Error for 25.0 percentile error |1.017199049249959 |
|Root Mean Square Error for 50.0 percentile error |3.0799378948907123|
|Root Mean Square Error for 75.0 percentile error |8.927414844545458 |
|Root Mean Square Error for 90.0 percentile error |23.51238189886229 |
|Root Mean Square Error for 100.0 percentile error|555.827300576082  |
+-------------------------------------------------+------------------+


WomenKurtas
+-------------------------------------------------+-----------------